In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import numpy as np
import random

In [16]:
df = pd.read_pickle(r'daten/otto_amazon_variation_pairs.pkl', compression='gzip')
df.head(10)

,amazon_product_name,otto_variation_name,variation_id,variation_asin,product_asin,amazon_product_url,file_name_timestamp,amazon_product_url_zyte,amazon_main_imageurl,amazon_description,...,pdh_variation_h_sk,pdh_product_h_sk,product_id,pbk,product_type,otto_product_url,otto_main_imageurl,kategorie_direct_bezeichnung,kategorie_bezeichnung,sub_kategorie_bezeichnung
0,Brother DCP-J1200W 3-in-1 Farbtintenstrahl-Mul...,"Brother DCP-J1200W Tintenstrahldrucker, (WLAN ...",1459585412,B0972RD6F5,B0972RD6F5,https://www.amazon.de/dp/B0972RD6F5/,2023-10-23 02:47:47+00:00,https://www.amazon.de/Brother-DCP-J1200W-Farbt...,https://m.media-amazon.com/images/I/41jxuc8HZs...,Produktbeschreibung des Herstellers ...,...,5d7afa8d68a6d88798dd37ed27093882,00b76adf49c4598d0a07b800e7958d36,1459585411,Drucker,Tintenstrahldrucker,https://otto.de/p/1459585411,https://i.otto.de/i/otto/bc4e3190-8618-54bf-b8...,Hardware,Multimedia & Toys,"Office Equipment, Mobile Computing & Digital I..."
1,AEG CCB544COBM 50 cm Standherd mit Glaskeramik...,AEG Elektro-Standherd CCB544COBM,1475311739,B0983HH8YJ,B0983HH8YJ,https://www.amazon.de/dp/B0983HH8YJ/,2023-11-10 01:13:44+00:00,https://www.amazon.de/AEG-CCB544COBM-Glaskeram...,https://m.media-amazon.com/images/I/71lHc-qSRT...,None,...,36643027eca54abd01e3d16fa0c3f313,0734c4795c54ca56c57c42174c79b9c2,1475296433,Herdsets,None,https://otto.de/p/1475296433,https://i.otto.de/i/otto/05f579c0-7391-5264-8e...,Hardware,Home Appliances,Built-In & Cooking
2,Gorenje GEC6C 40 WD Elektro-Standherd mit Glas...,"GORENJE Elektro-Standherd GEC6C40WD, Aqua Clean",1711068272,B0BHWDBXXM,B0BHWDBXXM,https://www.amazon.de/dp/B0BHWDBXXM/,2023-11-06 02:14:29+00:00,https://www.amazon.de/Gorenje-Elektro-Standher...,https://m.media-amazon.com/images/I/51a0ClIh7B...,None,...,4f89b91956d5ed60ec8f575046bc9130,3317acc1cfdceaa0a48e83e295a14fd1,1711068271,Herdsets,None,https://otto.de/p/1711068271,https://i.otto.de/i/otto/d5bb3b85-ce4e-5a08-a9...,Hardware,Home Appliances,Built-In & Cooking
3,Anita Damen Fleur Soft BH,Rosa Faia Schalen-BH Fleur (1-tlg) bequeme Sof...,1778134425,B00W6NBZ9O,B097S97F2Q,https://www.amazon.de/dp/B00W6NBZ9O,2023-11-10 09:02:56+00:00,https://www.amazon.de/Rosa-Faia-B%C3%BCgellose...,https://m.media-amazon.com/images/I/71PqlKBmHE...,Bügelloser T-Shirt-BH,...,ed4f06ee96fd8ce7f88077712e838fcb,9078a98c59f3f21c019759ee23c484c2,1778134419,BHs,Große Größen-BHs,https://otto.de/p/1778134419,https://i.otto.de/i/otto/dad3b553-ecda-51a0-9c...,LASCANA at OTTO,"CSI, Partner","Underwear CSI, Partner"
4,WENKO Türgarderobe Nostalgie - Türhängeleiste ...,"WENKO Türhaken Nostalgie, Türen, für einen Tür...",1065935236,B07CVYGJZF,B07CVYGJZF,https://www.amazon.de/dp/B07CVYGJZF/,2023-11-10 05:53:26+00:00,https://www.amazon.de/Wenko-T%C3%BCrkleiderhak...,https://m.media-amazon.com/images/I/51xRqBGHsN...,Produktbeschreibung des Herstellers ...,...,f68d088f35b3a88f04d66a4696c1bbc5,c757859dc5fc1ede70353963e73af1d7,1065935229,Haken,Garderobenhaken,https://otto.de/p/1065935229,https://i.otto.de/i/otto/3bd4b4c0-943d-593b-91...,Hardware,Garden DIY,Renovation
5,"WENKO Teleskop-System Herkules Duo, individuel...","WENKO Teleskopregal Herkules Duo, ausziehbar v...",1462235437,B07C7DPX6G,B07C7DPX6G,https://www.amazon.de/dp/B07C7DPX6G/,2023-11-10 05:53:26+00:00,https://www.amazon.de/WENKO-50611100-Teleskop-...,https://m.media-amazon.com/images/I/71cneCTpX3...,Produktbeschreibung des Herstellers ...,...,384075605abaacb0e13a4d05f3a53806,f6fa440327c8aa2a3484b5373c5feb8f,1462235436,Regale,Teleskopregale,https://otto.de/p/1462235436,https://i.otto.de/i/otto/b5af4d82-4ba4-536f-a6...,Hardware,Garden DIY,Renovation
6,"WENKO Teleskop-Stange Herkules, Ergänzungs-Sta...","WENKO Teleskopregal Herkules, (Hauptstange), 1...",1462235598,B07PH83595,B07PH83595,https://www.amazon.de/dp/B07PH83595/,2023-11-10 05:53:26+00:00,https://www.amazon.de/Hauptstange-Herkules-ver...,https://m.media-amazon.com/images/I/215s7AgrJA...,Produktbeschreibung des Herstellers ...,...,710294378e34caac

In [14]:
model = SentenceTransformer('all-MiniLM-L6-v2')

README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 679kB/s]


In [19]:
def negative_examples(df, num_negatives_per_positive=1):
    """
    Ähnliche Anzahl von negativen Beispielen, basierend auf den vorhandenen positiven Beispielen genrerien.
    Jedes negative Beispiel ist ein Paar von Produkten, die nicht übereinstimmen.
    
    :param df: DataFrame mit den positiven Beispielen.
    :param num_negatives_per_positive: Anzahl der negativen Beispiele, die pro positivem Beispiel generiert werden sollen.
    :return: DataFrame mit den negativen Beispielen.
    """
    negative_examples = []
    product_names = df['amazon_product_name'].tolist() + df['otto_variation_name'].tolist()
    
    for _, row in df.iterrows(): # der index selbst wird ignoriert 
        for _ in range(num_negatives_per_positive):
            # zuffälig zwei Produkte nehmen, die nicht paare sind 
            while True:
                product1 = random.choice(product_names)
                product2 = random.choice(product_names)
                
                # hierit wird sichergestellt, dass product1 und product2 nicht übereinstimmen und nicht in den positiven Beispielen sind
                if product1 != product2 and not ((df['amazon_product_name'] == product1) & (df['otto_variation_name'] == product2)).any():
                    break
            
            negative_examples.append({'amazon_product_name': product1, 'otto_variation_name': product2})
    
    return pd.DataFrame(negative_examples)

# schwere Negative bsp bauen!!!


In [15]:
positive_embeddings_1 = model.encode(df['amazon_product_name'].tolist())
positive_embeddings_2 = model.encode(df['otto_variation_name'].tolist())
# Absolute Differenz als Features berechnen 
positive_features = np.abs(positive_embeddings_1 - positive_embeddings_2)

In [20]:
negative_df = negative_examples(df)
negative_embeddings_1 = model.encode(negative_df['amazon_product_name'].tolist()) #eventuell nochmal mit batchsize machen
negative_embeddings_2 = model.encode(negative_df['otto_variation_name'].tolist())
negative_features = np.abs(negative_embeddings_1 - negative_embeddings_2)


In [21]:
# Features und Labels zusammenführen Positive Features
positive_labels = np.ones(len(positive_features))

In [22]:
# Features und Labels zusammenführen Negativ Features
negative_labels = np.zeros(len(negative_features))

In [32]:
# Features und Labels zusammenführen
X = np.concatenate([positive_features, negative_features], axis=0)
y = np.concatenate([positive_labels, negative_labels], axis=0)



In [33]:
# Trainings- und Testsets aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
X_train = np.array(X_train)
y_train = np.array(y_train)

print(y_train.shape)

(50654,)


In [34]:
# SVM trainieren
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [57]:
# Modell speichern
# from joblib import dump
# dump(clf, 'erste_svm_modell_19_feb_2024.joblib')


['erste_svm_modell_19_feb_2024.joblib']

In [35]:
# Vorhersagen und Evaluation 
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94     12527
         1.0       0.96      0.93      0.94     12801

    accuracy                           0.94     25328
   macro avg       0.94      0.94      0.94     25328
weighted avg       0.94      0.94      0.94     25328



In [41]:
# Modell und Daten laden
model = SentenceTransformer('all-MiniLM-L6-v2')
df = pd.read_excel(r'daten/amazon_otto_daten.xlsx')

In [42]:
ergebnisse = []

In [43]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [44]:
# Vorbereiten der Texte für die Batch-Verarbeitung
amazon_texts = df['amazon_product'].tolist()
otto_texts = [item for sublist in df[[f'otto_product_{i}' for i in range(1, 5)]].values.tolist() for item in sublist]

In [45]:
# Generieren der Embeddings in einem Batch Anzahl der Daten = 40 k =: 10 K Amazon x 4 Otto = 40 k time: 23m 40.5s
# ohne KNN:
# 10 K x 10 K = 100 K 
all_texts = amazon_texts + otto_texts
all_embeddings = model.encode(all_texts, batch_size=32)

In [47]:
# Aufteilen der Embeddings zurück in Amazon und Otto
amazon_embeddings = all_embeddings[:len(amazon_texts)]
otto_embeddings = np.array(all_embeddings[len(amazon_texts):]).reshape(-1, 4, all_embeddings.shape[1])

In [48]:
ergebnisse=[]

In [49]:
# optimierte Modell
for index, amazon_emb in enumerate(amazon_embeddings):
    distances = []
    for i in range(4):  # für jedes der 4 ähnlichsten Otto-Produkte
        otto_emb = otto_embeddings[index, i]
        
        # Feature für das Paar berechnen
        feature = np.abs(amazon_emb - otto_emb).reshape(1, -1)
        
        # Distanz zur Entscheidungsgrenze berechnen und normalisieren
        distance = clf.decision_function(feature)
        normalized_distance = sigmoid(distance)
        distances.append(normalized_distance)
    
    # ähnlichste Otto-Produkt bestimmen
    most_similar_index = np.argmax(distances)
    ergebnisse.append({
        'amazon_product': df.at[index, 'amazon_product'],
        'most_similar_otto_product': df.at[index, f'otto_product_{most_similar_index+1}'],
        'normalized_distance_score': distances[most_similar_index].item()  # Extrahieren des Skalarwerts
    })

In [40]:
#Modell ohne embedding, Laufzeit mit embedding = 60 min 
""" for index, row in df.iterrows():
    amazon_embedding = model.encode([row['amazon_product']])
    
    # Speichern Sie die Distanzen für jedes Otto-Produkt
    distances = []
    
    for i in range(1, 5):  # Für jedes der 4 ähnlichsten Otto-Produkte
        otto_embedding = model.encode([row[f'otto_product_{i}']])
        
        # Feature für das Paar berechnen
        feature = np.abs(amazon_embedding - otto_embedding).reshape(1, -1)
        
        # Distanz zur Entscheidungsgrenze berechnen
        distance = clf.decision_function(feature)
        normalized_distance = sigmoid(distance)
        distances.append(normalized_distance)
    
    # Bestimmen Sie das Otto-Produkt mit der geringsten Distanz zur Entscheidungsgrenze (höchste "Ähnlichkeit")
    most_similar_index = np.argmax(distances)  # Höchste Distanz, da größere Werte eine stärkere Zuordnung anzeigen
    most_similar_otto_product = row[f'otto_product_{most_similar_index+1}']  # +1, weil der Index bei 0 beginnt
    
    ergebnisse.append({
        'amazon_product': row['amazon_product'],
        'most_similar_otto_product': most_similar_otto_product,
        'distance_score': distances[most_similar_index][0]  # [0] Zugriff auf den Wert, wenn distance ein Array ist
    })
 """

ValueError: cannot reshape array of size 384 into shape (1,0)

In [50]:
ergebnisse_df = pd.DataFrame(ergebnisse)

In [51]:
ergebnisse_df.to_excel('daten/ergebnis_neu_optimiert.xlsx')

In [39]:
ergebnisse_df.head()

,amazon_product,most_similar_otto_product,distance_score
0,yann outdoorjacke outdoor wanderjacke mit kapu...,lkupro outdoorjacke damen herbst boucljacke tr...,4.911804
1,adidas terrex langarmshirt trace rocker jacked...,adidas originals outdoorjacke hooded premium l...,3.262285
2,yyv yogaleggings damen sporthose yogahosen spo...,fanyinshini sporthose yoga hosen frauen fake z...,2.009609
3,loud proud t shirt t shirt uni t shirts für m...,boss t shirt herren t shirt oberteil rundhals ...,1.242911
4,planam arbeitslatzhose latzhose tristep kornbl...,planam arbeitshose bundhose bundhose tristep k...,5.016873


In [52]:
ergebnisse_df.head()


,amazon_product,most_similar_otto_product,normalized_distance_score
0,yann outdoorjacke outdoor wanderjacke mit kapu...,lkupro outdoorjacke damen herbst boucljacke tr...,0.992695
1,adidas terrex langarmshirt trace rocker jacked...,adidas originals outdoorjacke hooded premium l...,0.963112
2,yyv yogaleggings damen sporthose yogahosen spo...,fanyinshini sporthose yoga hosen frauen fake z...,0.881802
3,loud proud t shirt t shirt uni t shirts für m...,boss t shirt herren t shirt oberteil rundhals ...,0.776070
4,planam arbeitslatzhose latzhose tristep kornbl...,planam arbeitshose bundhose bundhose tristep k...,0.993418


In [ ]:
for produkt in ergebnisse_df: 
    if row['normalized_distance']

In [64]:
gefilterte_produkte_df = ergebnisse_df[ergebnisse_df['normalized_distance_score'] < 0.9]
gefilterte_produkte_df.to_excel("daten/whitespots.xlsx", index=False)